# Acquiring Beer Data 

From Brewerydb: https://www.brewerydb.com/

In [ ]:
#import dependencies
import api_setup as ap
from tqdm import tqdm
import pandas as pd
from pprint import pprint

In [ ]:
beer_data = pd.DataFrame()
# withBreweries and withIngredients are completely empty
beer_features = ['id', 'name', 'nameDisplay', 'abv', 'styleId','year', 'glasswareId','isOrganic', 'breweries',
                'glasswareId', 'isRetired', 'status', 'statusDisplay','ingredients']

for feature in beer_features:
    beer_data[feature]=''

ing_col =['hops','malt','misc','yeast']
for ing in ing_col:
    beer_data[ing] = ''
count=0
for p in range(1,24):
   
    data = ap.get_data(page=p, custom='&withBreweries=Y&withIngredients=Y')
    try:
        for i in range(50):
          
            for feature in beer_features:
             #   print('feature ' + feature + ' value ' + str(data['data'][i][feature]))
                if feature == 'breweries':
                  
                    try:
                        beer_data.set_value(count,feature,data['data'][i][feature][0]['id'])
                    except Exception as e:
                        print ('no brewery')
                elif feature == 'ingredients':
                   
                    try:
                        beer_data.set_value(count,feature,len(data['data'][i][feature]))
                        #rint(data['data'][i][feature])
                        
                        for ing,val in data['data'][i][feature].items():
                            #print(ing)
                            
                                
                                    #print(ing)
                                    #rint(str(val[0]['id']))
                                    h_id_string =''
                                    for h_id in val:
                                        #print('hola')
                                        #print(h_id)
                                        try:
                                            h_id_string += str(h_id['id']) +';'
                                        except Exception as e:
                                          #  print ( h_id)
                                          #  print(data['data'][i][feature])
                                          #  print('error' + str(e))
                                             pass
                                    beer_data.set_value(count,ing,h_id_string)
                          
                                    
                    except Exception as e:
                       # print('error ' + str(e))
                        beer_data.set_value(count,feature,0)
                else:
                    try:
                        #print('stuff')
                        beer_data.set_value(count,feature,data['data'][i][feature])
                    except Exception as e: 
                        beer_data.set_value(count,feature,'n/a')
                        #print(f"name:{data['data'][i]['name']}, page:{p}, item:{i}, feature:{feature}, error:{e}")
                #print('end feature')
            count+=1
            #print(count)
            #beer_data.tail(2)
    except Exception as e:
        print("error " + str(e))
                      
                      

In [ ]:
beer_data.head()


In [ ]:
beer_data['breweries'].value_counts()

> we'll change the column names from camel case to snake case for better consistency in notebook

In [ ]:
beer_data.rename(columns={'id':'beer_id', 'nameDisplay':'display_name','isOrganic':'organic','isRetired':'retired','breweries':'brewery_id', 'styleId':'style_id', 'glasswareId':'glass_id'}, inplace=True)

In [ ]:
beer_data.drop('statusDisplay',axis=1, inplace=True)

In [ ]:
beer_data.head()

In [ ]:
beer_data.to_csv("data/beers.csv")

> we can see that there are 263 beers of the 1109 total beers that have a value for glass_id

In [ ]:
beer_glass = beer_data[beer_data["glass_id"].notna()]
print(beer_glass.shape)
beer_glass.head()

> Below, we can see the distribution of the glasses. The majority of the beers use glass5. Because of the lack of data in several other glass_id's, it would not be safe to generalize the data over other beers/glasses

In [ ]:
beer_glass.groupby('glass_id').count()['beer_id']

> before we can create our plots, we need to clean the data and make sure that the data types are consistent with the requirements for the plot. 

In [ ]:
# beer_glass.info() 
# shows that all variables are categorical and abv should be turned into a float and glass_id to int to be able to plot
beer_glass['abv'] = beer_glass['abv'].astype(float)
beer_glass['glass_id'] = beer_glass['glass_id'].astype(int)
beer_glass.info()

In [ ]:
abv_glass = beer_glass.boxplot('abv', "glass_id")

In [ ]:
abv_hist = beer_glass.plot(y='abv', kind='hist', bins=20, title='Alcohol by Volume', legend=False)